In [146]:
# %load /home/biswadip/autoreload_jn.py
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [147]:
# %load sentiment\ analysis-attention.py
#!/usr/bin/env python

# In[2]:
import os

# In[49]:


import numpy as np
import pandas as pd


# In[53]:


from models.text import CustomTokenizer
from models.model import word_list, max_sentences, maxlen

In [148]:
tokenizer = CustomTokenizer(word_list = word_list)

In [5]:
batch_s = 32

path = "negativeReviews/"
neg_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        neg_reviews.append(fl.read())
    

In [6]:
path = "positiveReviews//"
pos_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        pos_reviews.append(fl.read())

In [7]:
data = pd.DataFrame(
    {"text":neg_reviews, "sentiment":0}
).append(pd.DataFrame(
    {"text":pos_reviews, "sentiment":1}
))

print("Data Shape {}".format(data.shape))
# data.to_csv("tagged_data.csv")
print("Class Distribution {}".format(
    data.sentiment.value_counts())
)

Data Shape (25000, 2)
Class Distribution 1    12500
0    12500
Name: sentiment, dtype: int64


In [8]:
data = data.reset_index()

data = data.filter(["text","sentiment"])
data = data.sample(frac=1)

In [9]:
data = data[:100]

In [10]:
# =================================================
import tensorflow as tf

inp = tokenizer.doc_to_sequences(data.text.tolist())

In [11]:
inputs = []
for doc in inp:
    inputs.append(
        tf.keras.preprocessing.sequence.pad_sequences(
            doc, padding="post", value=0, maxlen=maxlen, dtype=None
        )
    )

In [12]:
a = np.zeros((len(inputs),max_sentences,maxlen))

for row,x in zip(a, inputs):
    row[:len(x)] = x[:50]

In [44]:
# Define Model

from models.model import get_model, ModelCheckpoint, max_sentences, maxlen
from models.model import HierarchicalAttentionLayer, SequenceAttentionLayer, AdditiveAttention
from models.data import Sequence_generator
# from models.tuner import tuner

In [16]:
model = tf.keras.models.load_model(
    "model/attention-wh_100_sh_600-01-0.79.h5",
    custom_objects={"HierarchicalAttentionLayer": HierarchicalAttentionLayer}
)

In [22]:
data.text.values[2]

'Sorry, I just didn\'t find the subject matter as compelling as the filmmaker did. The robot guy and the mole rat guy were pretty interesting, although Morris didn\'t really tell us much about them. The other two subjects were a bore. And the supposed "connections" between them didn\'t hold up.'

In [25]:
model.predict(a[1:2])

array([[0.13388208, 0.8661179 ]], dtype=float32)

In [34]:
hirer_layer = model.layers[1]

In [39]:
sentence_layer = hirer_layer.sentence_layer

In [41]:
embed, lstm, attention = sentence_layer.embed, sentence_layer.lstm, sentence_layer.attention_layer

In [46]:
# sentence_layer.embedding_len

In [92]:
rh= tf.reshape(
        embed(a[:2]), 
        shape = (-1 ,maxlen ,sentence_layer.embedding_len)
    )

In [93]:
rh.shape

TensorShape([100, 80, 50])

In [97]:
maxlen, max_sentences

(80, 50)

## Calculating Mask

In [129]:
embed_reshaped = tf.reshape(
        embed(a[:2]), 
        shape = (-1 ,maxlen ,sentence_layer.embedding_len)
    )

In [130]:
mask = tf.reshape(
    embed(a[:2])._keras_mask,
    shape = (-1, maxlen)
)

In [131]:
lstm_output = lstm(
    embed_reshaped, mask=mask
)

In [132]:
lstm_mask = lstm_output._keras_mask

In [124]:
# attention([lstm_output, lstm_output], mask = [lstm_mask, lstm_mask])

In [133]:
atten_scores = attention.get_attention_scores([lstm_output, lstm_output], mask = [lstm_mask, lstm_mask])

In [169]:
tokenizer.index_word[a[:2][0][2][2]]

'most'

In [166]:
# a[:2][0][3]

In [172]:
tf.argsort(atten_scores[0], direction="DESCENDING")

<tf.Tensor: shape=(80, 80), dtype=int32, numpy=
array([[12,  5,  6, ..., 77, 78, 79],
       [12,  5,  6, ..., 77, 78, 79],
       [12,  5,  6, ..., 77, 78, 79],
       ...,
       [12,  5,  6, ..., 77, 78, 79],
       [12,  5,  6, ..., 77, 78, 79],
       [12,  5,  6, ..., 77, 78, 79]], dtype=int32)>

In [170]:
# atten_scores[0][0]

In [171]:
# atten_scores[0][1]

In [150]:
data.text.values[0]

'In my humble opinion, this movie did not receive the recognition it deserved. Robert Redford lives near me here in Provo, Utah, at Sundance. I enjoy most of his work, and this was my favorite. I\'m sorry that more people didn\'t appreciate it. My grandmother was an avid reader and read the book years before it came out on the big screen. She gave it to me to read after we had seen the movie together. The movie and book hit an emotional spot within my heart, and I was weepy for several days after seeing the movie. Sometimes love isn\'t enough to keep our loved ones from hurting themselves. We see this in our own family relationships, yet our love and our families and their stories endure throughout generations of time. The cinematography was perfect and breathtaking -- I was awed by its beauty and how well it brought to life the words of the author of the book, Norman Maclean, "But when I am alone in the half light of the canyon, all existence seems to fade to a being with my soul, and